In [1]:
import tensorflow as tf

tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#changing the working directory

%cd /content/gdrive/My Drive/Kaggle


/content/gdrive/My Drive/Kaggle


In [4]:
% pwd

#Check the present working directory using pwd command

'/content/gdrive/My Drive/Kaggle'

In [5]:
!ls

 0.2.0.zip				  Recommendation_df_test1.csv
 Apple-Twitter-Sentiment-DFE.csv	  Recommendation_df_test.csv
 apple-twitter-sentiment-texts.csv	  Recommendation_df_train1.csv
 Bitcoin_prediction_One_hot_encoded.csv   Recommendation_df_train.csv
 BitCoinPrice.csv			  sampleSubmission.csv
 Bitcoin_tweet_sentiment.csv		  sampleSubmission.csv.zip
 BTCUSD_1hr.csv				  sampleSubmission_v2.csv.zip
 BTCUSD_1min.csv			  saved_models
 BTCUSD_day.csv				  Song_Recommendation_Model.h5
 Credit_Card_Autoencoder_Model.h5	  submission.csv
 creditcard.csv				  test.csv
 decoder_layer.png			  test_embed_df.csv
 decoder.png				  test.tsv
 encoder_layer.png			  train.csv
 encoder.png				  train_embed_df.csv
 FastText				  train-sample.csv.zip
'Harry Potter 1.csv'			  train.tsv
'Harry Potter 2.csv'			  train_v2.csv
'Harry Potter 3.csv'			  transformer.png
 kaggle.json				  tweetsfinal1.csv
 Movie_Conversation.csv			  tweetsfinal2.csv
 movie_conversations.txt		  tweetsfinal3.csv
 Movie_Dialogue_Model1.h5		  tw

In [6]:
# Tweets data from Oct 31 to Nov 9

import pandas as pd

header_list = ['Date', 'Tweet']
df1 = pd.read_csv('tweetsfinal1.csv', error_bad_lines=False, engine = 'python', names = header_list, header = None)
print(df1.shape)
df1.head()

Skipping line 845142: unexpected end of data


(845141, 2)


,Date,Tweet
0,Thu Nov 09 17:43:41 EST 2017,RT @Forbes: The Failure of SegWit2x Shows Bitc...
1,Thu Nov 09 17:43:40 EST 2017,RT @mindstatex: Lots of love from unknown mine...
2,Thu Nov 09 17:43:39 EST 2017,RT @FernandoHuamanX: Warning: Built-in Keylogg...
3,Thu Nov 09 17:43:39 EST 2017,RT @LevelNetwork: Join our telegram. All infor...
4,Thu Nov 09 17:43:39 EST 2017,"RT @realsheepwolf: $DIGAF: FLOAT=16M, THE ""ONL..."


In [7]:
df1 = df1[::-1]
df1

,Date,Tweet
845140,Tue Oct 31 00:00:33 EDT 2017,Bitcoin Cash cheapest on Cryptopia BTC-market:...
845139,Tue Oct 31 00:00:34 EDT 2017,Wall Street Hedge Funds Rush to Cryptocurrenci...
845138,Tue Oct 31 00:00:35 EDT 2017,Bitcoin value: $6132.78
845137,Tue Oct 31 00:00:35 EDT 2017,The price of bitcoin on @coinbase is now 6136....
845136,Tue Oct 31 00:00:36 EDT 2017,Current Prices:\n\n1 Waves = 3.46762$\n1 Waves...
...,...,...
4,Thu Nov 09 17:43:39 EST 2017,"RT @realsheepwolf: $DIGAF: FLOAT=16M, THE ""ONL..."
3,Thu Nov 09 17:43:39 EST 2017,RT @LevelNetwork: Join our telegram. All infor...
2,Thu Nov 09 17:43:39 EST 2017,RT @FernandoHuamanX: Warning: Built-in Keylogg...
1,Thu Nov 09 17:43:40 EST 2017,RT @mindstatex: Lots of love from unknown mine...


In [8]:
df1.isna().sum()

Date     0
Tweet    0
dtype: int64

In [9]:
df1.dtypes

Date     object
Tweet    object
dtype: object

In [10]:
# Function to preprocess text
import re

def preprocess(message):
  # Lowercase the twit message
  text = message.lower()
  # Replace URLs with a space in the message
  text = re.sub('https?:\/\/[a-zA-Z0-9@:%._\/+~#=?&;-]*', ' ', text)
  # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
  text = re.sub('\$[a-zA-Z0-9]*', ' ', text)
  # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
  text = re.sub('\@[a-zA-Z0-9]*', ' ', text)
  # Replace everything not a letter or apostrophe with a space
  text = re.sub('[^a-zA-Z\']', ' ', text)
  # Remove single letter words
  text = ' '.join( [w for w in text.split() if len(w)>1] )
    
  return text


df1['Tweet'] = df1['Tweet'].apply(preprocess)

df1.head()

,Date,Tweet
845140,Tue Oct 31 00:00:33 EDT 2017,bitcoin cash cheapest on cryptopia btc market ...
845139,Tue Oct 31 00:00:34 EDT 2017,wall street hedge funds rush to cryptocurrenci...
845138,Tue Oct 31 00:00:35 EDT 2017,bitcoin value
845137,Tue Oct 31 00:00:35 EDT 2017,the price of bitcoin on is now usd
845136,Tue Oct 31 00:00:36 EDT 2017,current prices waves waves btc waves wavesplat...


In [11]:
# Extract date without timezone

df1['Group'] = df1.Date.str.slice(8,10) + df1.Date.str.slice(11,13) + df1.Date.str.slice(14,16)

df1

,Date,Tweet,Group
845140,Tue Oct 31 00:00:33 EDT 2017,bitcoin cash cheapest on cryptopia btc market ...,310000
845139,Tue Oct 31 00:00:34 EDT 2017,wall street hedge funds rush to cryptocurrenci...,310000
845138,Tue Oct 31 00:00:35 EDT 2017,bitcoin value,310000
845137,Tue Oct 31 00:00:35 EDT 2017,the price of bitcoin on is now usd,310000
845136,Tue Oct 31 00:00:36 EDT 2017,current prices waves waves btc waves wavesplat...,310000
...,...,...,...
4,Thu Nov 09 17:43:39 EST 2017,rt float the only cryptocurrency exchange on t...,091743
3,Thu Nov 09 17:43:39 EST 2017,rt join our telegram all information about ico...,091743
2,Thu Nov 09 17:43:39 EST 2017,rt warning built in keylogger found in mantist...,091743
1,Thu Nov 09 17:43:40 EST 2017,rt lots of love from unknown miners miners are...,091743


In [12]:
df_grp1 = df1.groupby(['Group'], sort = False)['Tweet'].apply(' '.join).reset_index()
df_grp1.tail()

,Group,Tweet
14004,091739,rt great day for devcon and check out our toke...
14005,091740,bitcoin investors appear for the most part rel...
14006,091741,rt as bitcoin becomes popular encounter compar...
14007,091742,rt chart of the day there are bubbles then the...
14008,091743,rt usecase built on anti government but heavil...


In [13]:
df1.dtypes

Date     object
Tweet    object
Group    object
dtype: object

In [14]:
# Tweets data from Oct 31 to Nov 9

import pandas as pd

header_list = ['Date', 'Tweet']
df2 = pd.read_csv('tweetsfinal2.csv', error_bad_lines=False, engine = 'python', names = header_list, header = 0)
print(df2.shape)
df2.head()

(1055435, 2)


,Date,Tweet
0,Fri Nov 17 20:03:27 EST 2017,"1% #Bitcoin profits every 3 hours, every weekd..."
1,Fri Nov 17 20:03:27 EST 2017,What day will bitcoin break $10K??
2,https://t.co/UPmSbdKZ9F,None
3,Fri Nov 17 20:03:26 EST 2017,RT @SimplyVitalHQ: Our #HealthNexus technology...
4,Fri Nov 17 20:03:26 EST 2017,RT @BitcoinWrld: #Bitcoin Emerges as Crisis Cu...


In [15]:
df2 = df2[::-1]
df2

,Date,Tweet
1055434,Thu Nov 09 18:45:42 EST 2017,RT @CryptoJauregui: https://t.co/NAALJPUo8f Mo...
1055433,Thu Nov 09 18:45:42 EST 2017,RT @TheEconomist: People are buying Bitcoin be...
1055432,Thu Nov 09 18:45:43 EST 2017,RT @bitcoin_2x_org: AIRDROP will be closed on ...
1055431,Thu Nov 09 18:45:43 EST 2017,"A bit late, but interesting read nevertheless...."
1055430,https://t.co/oo7BCvHL9y #SegWit2x #Blockchain ...,None
...,...,...
4,Fri Nov 17 20:03:26 EST 2017,RT @BitcoinWrld: #Bitcoin Emerges as Crisis Cu...
3,Fri Nov 17 20:03:26 EST 2017,RT @SimplyVitalHQ: Our #HealthNexus technology...
2,https://t.co/UPmSbdKZ9F,None
1,Fri Nov 17 20:03:27 EST 2017,What day will bitcoin break $10K??


In [16]:
df2.isna().sum()

Date          0
Tweet    219369
dtype: int64

In [17]:
df2 = df2.dropna()
df2.head(20)

,Date,Tweet
1055434,Thu Nov 09 18:45:42 EST 2017,RT @CryptoJauregui: https://t.co/NAALJPUo8f Mo...
1055433,Thu Nov 09 18:45:42 EST 2017,RT @TheEconomist: People are buying Bitcoin be...
1055432,Thu Nov 09 18:45:43 EST 2017,RT @bitcoin_2x_org: AIRDROP will be closed on ...
1055431,Thu Nov 09 18:45:43 EST 2017,"A bit late, but interesting read nevertheless...."
1055429,Thu Nov 09 18:45:44 EST 2017,RT @JWPopeo: #Bitcoin expansion off the table
1055428,Thu Nov 09 18:45:44 EST 2017,"RT @alistairmilne: You can build a fast, effic..."
1055427,Thu Nov 09 18:45:45 EST 2017,"Once An Underground Currency, Bitcoin Emerges ..."
1055426,Thu Nov 09 18:45:46 EST 2017,RT @STKtoken: We're @CryptoMania_'s Coin of th...
1055425,Thu Nov 09 18:45:47 EST 2017,RT @swissborg: SwissBorg CEO Cyrus Fazel for @...
1055424,Thu Nov 09 18:45:47 EST 2017,Millennials often get the blame for ruining tr...


In [18]:
df2['Tweet'] = df2['Tweet'].apply(preprocess)

df2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,Tweet
1055434,Thu Nov 09 18:45:42 EST 2017,rt more charges filed against trader who used ...
1055433,Thu Nov 09 18:45:42 EST 2017,rt people are buying bitcoin because they expe...
1055432,Thu Nov 09 18:45:43 EST 2017,rt org airdrop will be closed on at pm we will...
1055431,Thu Nov 09 18:45:43 EST 2017,bit late but interesting read nevertheless
1055429,Thu Nov 09 18:45:44 EST 2017,rt bitcoin expansion off the table


In [19]:
df2['Group'] = df2.Date.str.slice(8,10) + df2.Date.str.slice(11,13) + df2.Date.str.slice(14,16)

df2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,Tweet,Group
1055434,Thu Nov 09 18:45:42 EST 2017,rt more charges filed against trader who used ...,091845
1055433,Thu Nov 09 18:45:42 EST 2017,rt people are buying bitcoin because they expe...,091845
1055432,Thu Nov 09 18:45:43 EST 2017,rt org airdrop will be closed on at pm we will...,091845
1055431,Thu Nov 09 18:45:43 EST 2017,bit late but interesting read nevertheless,091845
1055429,Thu Nov 09 18:45:44 EST 2017,rt bitcoin expansion off the table,091845
...,...,...,...
5,Fri Nov 17 20:03:25 EST 2017,rt bitcoin price up or so since jaime dimon ca...,172003
4,Fri Nov 17 20:03:26 EST 2017,rt bitcoin emerges as crisis currency in hotsp...,172003
3,Fri Nov 17 20:03:26 EST 2017,rt our healthnexus technology is already chang...,172003
1,Fri Nov 17 20:03:27 EST 2017,what day will bitcoin break,172003


In [20]:
df_grp2 = df2.groupby(['Group'], sort = False)['Tweet'].apply(' '.join).reset_index()
df_grp2.head()

,Group,Tweet
0,091845,rt more charges filed against trader who used ...
1,091846,rt as bitcoin becomes popular encounter compar...
2,091847,rt hedge funds are ignoring jamie dimon and pi...
3,091848,rt as bitcoin becomes popular encounter compar...
4,091849,rt as bitcoin becomes popular encounter compar...


In [21]:
df_grp2.tail()

,Group,Tweet
11594,171959,just the beginning what the tezos lawsuits mea...
11595,172000,bad news bitcoin on the decline bitcoin btc pr...
11596,172001,korea hyosung now supports bitcoin at atms wil...
11597,172002,premir bitcoin paga via rt maximum rate up to ...
11598,172003,anyone else feeling beartrap happening with bt...


In [22]:
# Tweets data from Oct 31 to Nov 9

import pandas as pd

header_list2 = ['Date', 'Tweet' , 'None1', 'None2']
df3 = pd.read_csv('tweetsfinal3.csv', error_bad_lines=False, engine = 'python', names = header_list2, header = None)
print(df3.shape)
df3.head()

Skipping line 352574: NULL byte detected. This byte cannot be processed in Python's native csv library at the moment, so please pass in engine='c' instead
Skipping line 353631: NULL byte detected. This byte cannot be processed in Python's native csv library at the moment, so please pass in engine='c' instead
Skipping line 883146: unexpected end of data


(883143, 4)


,Date,Tweet,None1,None2
0,Mon Nov 27 17:40:38 EST 2017,"RT @daneelproject: Another day, another high s...",NaN,NaN
1,Mon Nov 27 17:40:38 EST 2017,RT @BankExProtocol: Be prepared to BKX #tokens...,NaN,NaN
2,Mon Nov 27 17:40:38 EST 2017,(Bitcoin Price Analysis: Bitcoin Tests Parabol...,NaN,NaN
3,Mon Nov 27 17:40:37 EST 2017,RT @daneelproject: Daneel uses a specially dev...,NaN,NaN
4,Mon Nov 27 17:40:36 EST 2017,RT @JacBurns_Comext: #Blockchain & Energy: her...,NaN,NaN


In [23]:
df3 = df3[::-1]
df3

,Date,Tweet,None1,None2
883142,Mon Nov 20 13:40:16 EST 2017,CME's Bitcoin Futures Likely to Start Trading ...,NaN,NaN
883141,Mon Nov 20 13:40:16 EST 2017,CME's Bitcoin Futures Likely to Start Trading ...,NaN,NaN
883140,Mon Nov 20 13:40:16 EST 2017,#CheckItOut as #AML bitcoin looks to test the ...,NaN,NaN
883139,Mon Nov 20 13:40:16 EST 2017,RT @charliebilello: Bitcoin's market cap just ...,NaN,NaN
883138,Mon Nov 20 13:40:17 EST 2017,"Bitcoin continues to crush it, breaks through ...",NaN,NaN
...,...,...,...,...
4,Mon Nov 27 17:40:36 EST 2017,RT @JacBurns_Comext: #Blockchain & Energy: her...,NaN,NaN
3,Mon Nov 27 17:40:37 EST 2017,RT @daneelproject: Daneel uses a specially dev...,NaN,NaN
2,Mon Nov 27 17:40:38 EST 2017,(Bitcoin Price Analysis: Bitcoin Tests Parabol...,NaN,NaN
1,Mon Nov 27 17:40:38 EST 2017,RT @BankExProtocol: Be prepared to BKX #tokens...,NaN,NaN


In [24]:
df3.isna().sum()

Date          0
Tweet         0
None1    882897
None2    882897
dtype: int64

In [25]:
df3 = df3.drop(['None1','None2'], axis = 1)
df3

,Date,Tweet
883142,Mon Nov 20 13:40:16 EST 2017,CME's Bitcoin Futures Likely to Start Trading ...
883141,Mon Nov 20 13:40:16 EST 2017,CME's Bitcoin Futures Likely to Start Trading ...
883140,Mon Nov 20 13:40:16 EST 2017,#CheckItOut as #AML bitcoin looks to test the ...
883139,Mon Nov 20 13:40:16 EST 2017,RT @charliebilello: Bitcoin's market cap just ...
883138,Mon Nov 20 13:40:17 EST 2017,"Bitcoin continues to crush it, breaks through ..."
...,...,...
4,Mon Nov 27 17:40:36 EST 2017,RT @JacBurns_Comext: #Blockchain & Energy: her...
3,Mon Nov 27 17:40:37 EST 2017,RT @daneelproject: Daneel uses a specially dev...
2,Mon Nov 27 17:40:38 EST 2017,(Bitcoin Price Analysis: Bitcoin Tests Parabol...
1,Mon Nov 27 17:40:38 EST 2017,RT @BankExProtocol: Be prepared to BKX #tokens...


In [26]:
df3.isna().sum()

Date     0
Tweet    0
dtype: int64

In [27]:
df3.dtypes

Date     object
Tweet    object
dtype: object

In [28]:
df3['Tweet'] = df3['Tweet'].apply(preprocess)

df3.head()

,Date,Tweet
883142,Mon Nov 20 13:40:16 EST 2017,cme's bitcoin futures likely to start trading ...
883141,Mon Nov 20 13:40:16 EST 2017,cme's bitcoin futures likely to start trading ...
883140,Mon Nov 20 13:40:16 EST 2017,checkitout as aml bitcoin looks to test the wa...
883139,Mon Nov 20 13:40:16 EST 2017,rt bitcoin's market cap just passed mcdonald's...
883138,Mon Nov 20 13:40:17 EST 2017,bitcoin continues to crush it breaks through


In [29]:
# Extract date without timezone

df3['Group'] = df3.Date.str.slice(8,10) + df3.Date.str.slice(11,13) + df3.Date.str.slice(14,16)

df3

,Date,Tweet,Group
883142,Mon Nov 20 13:40:16 EST 2017,cme's bitcoin futures likely to start trading ...,201340
883141,Mon Nov 20 13:40:16 EST 2017,cme's bitcoin futures likely to start trading ...,201340
883140,Mon Nov 20 13:40:16 EST 2017,checkitout as aml bitcoin looks to test the wa...,201340
883139,Mon Nov 20 13:40:16 EST 2017,rt bitcoin's market cap just passed mcdonald's...,201340
883138,Mon Nov 20 13:40:17 EST 2017,bitcoin continues to crush it breaks through,201340
...,...,...,...
4,Mon Nov 27 17:40:36 EST 2017,rt comext blockchain energy here are the most ...,271740
3,Mon Nov 27 17:40:37 EST 2017,rt daneel uses specially developed algorithm t...,271740
2,Mon Nov 27 17:40:38 EST 2017,bitcoin price analysis bitcoin tests parabolic...,271740
1,Mon Nov 27 17:40:38 EST 2017,rt be prepared to bkx tokensale answer about w...,271740


In [30]:
df_grp3 = df3.groupby(['Group'], sort = False)['Tweet'].apply(' '.join).reset_index()
df_grp3

,Group,Tweet
0,201340,cme's bitcoin futures likely to start trading ...
1,201341,rt bitcoin's market cap is now as huge as mcdo...
2,201342,rt aigang ico overview find out more at aigang...
3,201343,rt as predicted bitcoin surges past to new all...
4,201344,rt comext how bitcoin transactions work blockc...
...,...,...
10316,271736,reason to consider ico this holiday season we ...
10317,271737,rt crypto current price of bitcoin is rt bitco...
10318,271738,stickac latest firmware is no different to old...
10319,271739,here's maybe strange question but think it's o...


In [31]:
# df_grp3['Tweet'][0]

In [32]:
df_grp1  # from 31 Oct - 00:00 to 9 nov 17:43

,Group,Tweet
0,310000,bitcoin cash cheapest on cryptopia btc market ...
1,310001,current bitcoin price gbp btc quantum attacks ...
2,310002,rt participate in the dalecoin second round bo...
3,310003,rt what is bitconnect quick start guide to bit...
4,310004,marketalert last top price change avg of the t...
...,...,...
14004,091739,rt great day for devcon and check out our toke...
14005,091740,bitcoin investors appear for the most part rel...
14006,091741,rt as bitcoin becomes popular encounter compar...
14007,091742,rt chart of the day there are bubbles then the...


In [33]:
df_grp2   # from 9 nov - 18:45 to 17 nov 20:03

,Group,Tweet
0,091845,rt more charges filed against trader who used ...
1,091846,rt as bitcoin becomes popular encounter compar...
2,091847,rt hedge funds are ignoring jamie dimon and pi...
3,091848,rt as bitcoin becomes popular encounter compar...
4,091849,rt as bitcoin becomes popular encounter compar...
...,...,...
11594,171959,just the beginning what the tezos lawsuits mea...
11595,172000,bad news bitcoin on the decline bitcoin btc pr...
11596,172001,korea hyosung now supports bitcoin at atms wil...
11597,172002,premir bitcoin paga via rt maximum rate up to ...


In [34]:
df_grp3  # from 20 nov - 13:40 to 27 nov 17:40

,Group,Tweet
0,201340,cme's bitcoin futures likely to start trading ...
1,201341,rt bitcoin's market cap is now as huge as mcdo...
2,201342,rt aigang ico overview find out more at aigang...
3,201343,rt as predicted bitcoin surges past to new all...
4,201344,rt comext how bitcoin transactions work blockc...
...,...,...
10316,271736,reason to consider ico this holiday season we ...
10317,271737,rt crypto current price of bitcoin is rt bitco...
10318,271738,stickac latest firmware is no different to old...
10319,271739,here's maybe strange question but think it's o...


In [35]:
df_bc = pd.read_csv('BTCUSD_1min.csv')
df_bc

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1586390400000,2020-04-09 00:00:00,BTCUSD,7369.60,7369.60,7348.47,7350.41,7.733528
1,1586390340000,2020-04-08 23:59:00,BTCUSD,7364.37,7369.60,7364.37,7369.60,1.310954
2,1586390280000,2020-04-08 23:58:00,BTCUSD,7360.89,7367.18,7360.89,7364.37,1.018774
3,1586390220000,2020-04-08 23:57:00,BTCUSD,7357.62,7366.12,7357.62,7360.89,0.026251
4,1586390160000,2020-04-08 23:56:00,BTCUSD,7370.71,7370.71,7357.62,7357.62,0.444867
...,...,...,...,...,...,...,...,...
2283514,1444311840,2015-10-08 13:44:00,BTCUSD,242.96,242.96,242.96,242.96,0.033491
2283515,1444311780,2015-10-08 13:43:00,BTCUSD,242.95,242.96,242.95,242.96,0.010000
2283516,1444311720,2015-10-08 13:42:00,BTCUSD,242.95,242.95,242.95,242.95,0.000000
2283517,1444311660,2015-10-08 13:41:00,BTCUSD,242.50,242.95,242.50,242.95,0.001000


In [36]:
df_bc = df_bc[::-1]
df_bc

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
2283518,1444311600,2015-10-08 13:40:00,BTCUSD,0.00,242.50,0.00,242.50,0.050000
2283517,1444311660,2015-10-08 13:41:00,BTCUSD,242.50,242.95,242.50,242.95,0.001000
2283516,1444311720,2015-10-08 13:42:00,BTCUSD,242.95,242.95,242.95,242.95,0.000000
2283515,1444311780,2015-10-08 13:43:00,BTCUSD,242.95,242.96,242.95,242.96,0.010000
2283514,1444311840,2015-10-08 13:44:00,BTCUSD,242.96,242.96,242.96,242.96,0.033491
...,...,...,...,...,...,...,...,...
4,1586390160000,2020-04-08 23:56:00,BTCUSD,7370.71,7370.71,7357.62,7357.62,0.444867
3,1586390220000,2020-04-08 23:57:00,BTCUSD,7357.62,7366.12,7357.62,7360.89,0.026251
2,1586390280000,2020-04-08 23:58:00,BTCUSD,7360.89,7367.18,7360.89,7364.37,1.018774
1,1586390340000,2020-04-08 23:59:00,BTCUSD,7364.37,7369.60,7364.37,7369.60,1.310954


In [37]:
df_bc.dtypes

Unix Timestamp      int64
Date               object
Symbol             object
Open              float64
High              float64
Low               float64
Close             float64
Volume            float64
dtype: object

In [38]:
import pandas as pd
df_bc['Date_new'] = pd.to_datetime(df_bc['Date'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [39]:
df_bc.tail(20)

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,Date_new
19,1586389260000,2020-04-08 23:41:00,BTCUSD,7387.13,7387.13,7387.13,7387.13,0.055430,2020-04-08 23:41:00
18,1586389320000,2020-04-08 23:42:00,BTCUSD,7387.13,7392.10,7387.13,7392.09,0.000009,2020-04-08 23:42:00
17,1586389380000,2020-04-08 23:43:00,BTCUSD,7392.09,7392.92,7384.80,7385.72,0.037175,2020-04-08 23:43:00
16,1586389440000,2020-04-08 23:44:00,BTCUSD,7385.72,7387.68,7378.44,7385.03,0.085717,2020-04-08 23:44:00
15,1586389500000,2020-04-08 23:45:00,BTCUSD,7385.03,7387.02,7380.42,7380.42,0.000062,2020-04-08 23:45:00
14,1586389560000,2020-04-08 23:46:00,BTCUSD,7380.42,7380.69,7374.26,7374.26,0.673916,2020-04-08 23:46:00
13,1586389620000,2020-04-08 23:47:00,BTCUSD,7374.26,7378.83,7371.39,7371.39,0.377906,2020-04-08 23:47:00
12,1586389680000,2020-04-08 23:48:00,BTCUSD,7371.39,7376.37,7366.17,7376.28,0.110318,2020-04-08 23:48:00
11,1586389740000,2020-04-08 23:49:00,BTCUSD,7376.28,7376.28,7355.00,7361.43,3.404130,2020-04-08 23:49:00
10,1586389800000,2020-04-08 23:50:00,BTCUSD,7361.43,7363.60,7359.38,7363.57,0.000033,2020-04-08 23:50:00


In [40]:
df_bc.dtypes

Unix Timestamp             int64
Date                      object
Symbol                    object
Open                     float64
High                     float64
Low                      float64
Close                    float64
Volume                   float64
Date_new          datetime64[ns]
dtype: object

In [41]:
df_bc = df_bc.drop(['Unix Timestamp', 'Symbol', 'Open','High','Low'], axis = 1)
df_bc

,Date,Close,Volume,Date_new
2283518,2015-10-08 13:40:00,242.50,0.050000,2015-10-08 13:40:00
2283517,2015-10-08 13:41:00,242.95,0.001000,2015-10-08 13:41:00
2283516,2015-10-08 13:42:00,242.95,0.000000,2015-10-08 13:42:00
2283515,2015-10-08 13:43:00,242.96,0.010000,2015-10-08 13:43:00
2283514,2015-10-08 13:44:00,242.96,0.033491,2015-10-08 13:44:00
...,...,...,...,...
4,2020-04-08 23:56:00,7357.62,0.444867,2020-04-08 23:56:00
3,2020-04-08 23:57:00,7360.89,0.026251,2020-04-08 23:57:00
2,2020-04-08 23:58:00,7364.37,1.018774,2020-04-08 23:58:00
1,2020-04-08 23:59:00,7369.60,1.310954,2020-04-08 23:59:00


In [42]:
type(df_bc['Date_new'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [43]:
temp1 = df_bc[(df_bc['Date_new'] >= '2017-10-31 00:00:00') & (df_bc['Date_new'] <= '2017-11-09 17:43:00')]
temp1

,Date,Close,Volume,Date_new
1198578,2017-10-31 00:00:00,6124.68,0.134906,2017-10-31 00:00:00
1198577,2017-10-31 00:01:00,6124.68,1.223548,2017-10-31 00:01:00
1198576,2017-10-31 00:02:00,6124.69,3.294787,2017-10-31 00:02:00
1198575,2017-10-31 00:03:00,6124.68,0.110868,2017-10-31 00:03:00
1198574,2017-10-31 00:04:00,6124.69,0.087178,2017-10-31 00:04:00
...,...,...,...,...
1184559,2017-11-09 17:39:00,7186.48,5.741724,2017-11-09 17:39:00
1184558,2017-11-09 17:40:00,7187.34,0.647699,2017-11-09 17:40:00
1184557,2017-11-09 17:41:00,7187.34,0.124132,2017-11-09 17:41:00
1184556,2017-11-09 17:42:00,7190.80,5.483823,2017-11-09 17:42:00


In [44]:
# Extract date without timezone

temp1['Group'] = temp1.Date.str.slice(8,10) + temp1.Date.str.slice(11,13) + temp1.Date.str.slice(14,16)

temp1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Date,Close,Volume,Date_new,Group
1198578,2017-10-31 00:00:00,6124.68,0.134906,2017-10-31 00:00:00,310000
1198577,2017-10-31 00:01:00,6124.68,1.223548,2017-10-31 00:01:00,310001
1198576,2017-10-31 00:02:00,6124.69,3.294787,2017-10-31 00:02:00,310002
1198575,2017-10-31 00:03:00,6124.68,0.110868,2017-10-31 00:03:00,310003
1198574,2017-10-31 00:04:00,6124.69,0.087178,2017-10-31 00:04:00,310004
...,...,...,...,...,...
1184559,2017-11-09 17:39:00,7186.48,5.741724,2017-11-09 17:39:00,091739
1184558,2017-11-09 17:40:00,7187.34,0.647699,2017-11-09 17:40:00,091740
1184557,2017-11-09 17:41:00,7187.34,0.124132,2017-11-09 17:41:00,091741
1184556,2017-11-09 17:42:00,7190.80,5.483823,2017-11-09 17:42:00,091742


In [45]:
a = pd.merge(temp1, df_grp1, how='outer', suffixes=('','_y'), indicator=True)
a


,Date,Close,Volume,Date_new,Group,Tweet,_merge
0,2017-10-31 00:00:00,6124.68,0.134906,2017-10-31 00:00:00,310000,bitcoin cash cheapest on cryptopia btc market ...,both
1,2017-10-31 00:01:00,6124.68,1.223548,2017-10-31 00:01:00,310001,current bitcoin price gbp btc quantum attacks ...,both
2,2017-10-31 00:02:00,6124.69,3.294787,2017-10-31 00:02:00,310002,rt participate in the dalecoin second round bo...,both
3,2017-10-31 00:03:00,6124.68,0.110868,2017-10-31 00:03:00,310003,rt what is bitconnect quick start guide to bit...,both
4,2017-10-31 00:04:00,6124.69,0.087178,2017-10-31 00:04:00,310004,marketalert last top price change avg of the t...,both
...,...,...,...,...,...,...,...
14019,2017-11-09 17:39:00,7186.48,5.741724,2017-11-09 17:39:00,091739,rt great day for devcon and check out our toke...,both
14020,2017-11-09 17:40:00,7187.34,0.647699,2017-11-09 17:40:00,091740,bitcoin investors appear for the most part rel...,both
14021,2017-11-09 17:41:00,7187.34,0.124132,2017-11-09 17:41:00,091741,rt as bitcoin becomes popular encounter compar...,both
14022,2017-11-09 17:42:00,7190.80,5.483823,2017-11-09 17:42:00,091742,rt chart of the day there are bubbles then the...,both


In [46]:
rows_in_temp1_not_in_df_grp1 = a[a['_merge']=='left_only'][a.columns]
rows_in_temp1_not_in_df_grp1

,Date,Close,Volume,Date_new,Group,Tweet,_merge
6788,2017-11-04 17:08:00,7347.76,0.166333,2017-11-04 17:08:00,041708,NaN,left_only
6794,2017-11-04 17:14:00,7353.65,0.143322,2017-11-04 17:14:00,041714,NaN,left_only
6795,2017-11-04 17:15:00,7353.63,2.068931,2017-11-04 17:15:00,041715,NaN,left_only
6796,2017-11-04 17:16:00,7353.64,0.219322,2017-11-04 17:16:00,041716,NaN,left_only
6802,2017-11-04 17:22:00,7338.62,1.187626,2017-11-04 17:22:00,041722,NaN,left_only
6807,2017-11-04 17:27:00,7311.54,7.882891,2017-11-04 17:27:00,041727,NaN,left_only
6814,2017-11-04 17:34:00,7322.16,1.353016,2017-11-04 17:34:00,041734,NaN,left_only
6815,2017-11-04 17:35:00,7323.39,2.867828,2017-11-04 17:35:00,041735,NaN,left_only
6816,2017-11-04 17:36:00,7322.44,0.277664,2017-11-04 17:36:00,041736,NaN,left_only
6817,2017-11-04 17:37:00,7322.44,0.000000,2017-11-04 17:37:00,041737,NaN,left_only


In [47]:
temp1 = pd.merge(df_grp1, temp1, how='inner', on=['Group'])
temp1 = temp1.drop(['Tweet'], axis = 1)
temp1

,Group,Date,Close,Volume,Date_new
0,310000,2017-10-31 00:00:00,6124.68,0.134906,2017-10-31 00:00:00
1,310001,2017-10-31 00:01:00,6124.68,1.223548,2017-10-31 00:01:00
2,310002,2017-10-31 00:02:00,6124.69,3.294787,2017-10-31 00:02:00
3,310003,2017-10-31 00:03:00,6124.68,0.110868,2017-10-31 00:03:00
4,310004,2017-10-31 00:04:00,6124.69,0.087178,2017-10-31 00:04:00
...,...,...,...,...,...
14004,091739,2017-11-09 17:39:00,7186.48,5.741724,2017-11-09 17:39:00
14005,091740,2017-11-09 17:40:00,7187.34,0.647699,2017-11-09 17:40:00
14006,091741,2017-11-09 17:41:00,7187.34,0.124132,2017-11-09 17:41:00
14007,091742,2017-11-09 17:42:00,7190.80,5.483823,2017-11-09 17:42:00


In [48]:
# from 9 nov - 18:45 to 17 nov 20:03  11599

In [49]:
temp2 = df_bc[(df_bc['Date_new'] >= '2017-11-09 18:45:00') & (df_bc['Date_new'] <= '2017-11-17 20:03:00')]
temp2

,Date,Close,Volume,Date_new
1184493,2017-11-09 18:45:00,7240.00,1.881148,2017-11-09 18:45:00
1184492,2017-11-09 18:46:00,7238.01,8.115839,2017-11-09 18:46:00
1184491,2017-11-09 18:47:00,7200.50,10.694437,2017-11-09 18:47:00
1184490,2017-11-09 18:48:00,7198.92,3.658408,2017-11-09 18:48:00
1184489,2017-11-09 18:49:00,7198.92,9.340732,2017-11-09 18:49:00
...,...,...,...,...
1172899,2017-11-17 19:59:00,7821.01,3.275958,2017-11-17 19:59:00
1172898,2017-11-17 20:00:00,7821.00,1.722127,2017-11-17 20:00:00
1172897,2017-11-17 20:01:00,7805.01,0.067277,2017-11-17 20:01:00
1172896,2017-11-17 20:02:00,7804.99,2.055784,2017-11-17 20:02:00


In [50]:
# Extract date without timezone

temp2['Group'] = temp2.Date.str.slice(8,10) + temp2.Date.str.slice(11,13) + temp2.Date.str.slice(14,16)

temp2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Date,Close,Volume,Date_new,Group
1184493,2017-11-09 18:45:00,7240.00,1.881148,2017-11-09 18:45:00,091845
1184492,2017-11-09 18:46:00,7238.01,8.115839,2017-11-09 18:46:00,091846
1184491,2017-11-09 18:47:00,7200.50,10.694437,2017-11-09 18:47:00,091847
1184490,2017-11-09 18:48:00,7198.92,3.658408,2017-11-09 18:48:00,091848
1184489,2017-11-09 18:49:00,7198.92,9.340732,2017-11-09 18:49:00,091849
...,...,...,...,...,...
1172899,2017-11-17 19:59:00,7821.01,3.275958,2017-11-17 19:59:00,171959
1172898,2017-11-17 20:00:00,7821.00,1.722127,2017-11-17 20:00:00,172000
1172897,2017-11-17 20:01:00,7805.01,0.067277,2017-11-17 20:01:00,172001
1172896,2017-11-17 20:02:00,7804.99,2.055784,2017-11-17 20:02:00,172002


In [51]:
temp3 = df_bc[(df_bc['Date_new'] >= '2017-11-20 13:40:00') & (df_bc['Date_new'] <= '2017-11-27 17:40:00')]  # from 20 nov - 13:40 to 27 nov 17:40  - 10321
temp3

,Date,Close,Volume,Date_new
1168958,2017-11-20 13:40:00,8147.59,0.356399,2017-11-20 13:40:00
1168957,2017-11-20 13:41:00,8147.30,0.122434,2017-11-20 13:41:00
1168956,2017-11-20 13:42:00,8147.32,0.489542,2017-11-20 13:42:00
1168955,2017-11-20 13:43:00,8158.14,5.062910,2017-11-20 13:43:00
1168954,2017-11-20 13:44:00,8158.13,12.709487,2017-11-20 13:44:00
...,...,...,...,...
1158642,2017-11-27 17:36:00,9494.66,11.753007,2017-11-27 17:36:00
1158641,2017-11-27 17:37:00,9496.96,1.201738,2017-11-27 17:37:00
1158640,2017-11-27 17:38:00,9496.85,0.499032,2017-11-27 17:38:00
1158639,2017-11-27 17:39:00,9496.85,0.007648,2017-11-27 17:39:00


In [52]:
# Extract date without timezone

temp3['Group'] = temp3.Date.str.slice(8,10) + temp3.Date.str.slice(11,13) + temp3.Date.str.slice(14,16)

temp3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Date,Close,Volume,Date_new,Group
1168958,2017-11-20 13:40:00,8147.59,0.356399,2017-11-20 13:40:00,201340
1168957,2017-11-20 13:41:00,8147.30,0.122434,2017-11-20 13:41:00,201341
1168956,2017-11-20 13:42:00,8147.32,0.489542,2017-11-20 13:42:00,201342
1168955,2017-11-20 13:43:00,8158.14,5.062910,2017-11-20 13:43:00,201343
1168954,2017-11-20 13:44:00,8158.13,12.709487,2017-11-20 13:44:00,201344
...,...,...,...,...,...
1158642,2017-11-27 17:36:00,9494.66,11.753007,2017-11-27 17:36:00,271736
1158641,2017-11-27 17:37:00,9496.96,1.201738,2017-11-27 17:37:00,271737
1158640,2017-11-27 17:38:00,9496.85,0.499032,2017-11-27 17:38:00,271738
1158639,2017-11-27 17:39:00,9496.85,0.007648,2017-11-27 17:39:00,271739


In [53]:
tweets_final = pd.concat([df_grp1, df_grp2, df_grp3])
tweets_final

,Group,Tweet
0,310000,bitcoin cash cheapest on cryptopia btc market ...
1,310001,current bitcoin price gbp btc quantum attacks ...
2,310002,rt participate in the dalecoin second round bo...
3,310003,rt what is bitconnect quick start guide to bit...
4,310004,marketalert last top price change avg of the t...
...,...,...
10316,271736,reason to consider ico this holiday season we ...
10317,271737,rt crypto current price of bitcoin is rt bitco...
10318,271738,stickac latest firmware is no different to old...
10319,271739,here's maybe strange question but think it's o...


In [54]:
bc_final = pd.concat([temp1, temp2, temp3])
bc_final

,Group,Date,Close,Volume,Date_new
0,310000,2017-10-31 00:00:00,6124.68,0.134906,2017-10-31 00:00:00
1,310001,2017-10-31 00:01:00,6124.68,1.223548,2017-10-31 00:01:00
2,310002,2017-10-31 00:02:00,6124.69,3.294787,2017-10-31 00:02:00
3,310003,2017-10-31 00:03:00,6124.68,0.110868,2017-10-31 00:03:00
4,310004,2017-10-31 00:04:00,6124.69,0.087178,2017-10-31 00:04:00
...,...,...,...,...,...
1158642,271736,2017-11-27 17:36:00,9494.66,11.753007,2017-11-27 17:36:00
1158641,271737,2017-11-27 17:37:00,9496.96,1.201738,2017-11-27 17:37:00
1158640,271738,2017-11-27 17:38:00,9496.85,0.499032,2017-11-27 17:38:00
1158639,271739,2017-11-27 17:39:00,9496.85,0.007648,2017-11-27 17:39:00


In [55]:
bc_final['Increase'] = bc_final['Close'].pct_change().apply(lambda x: 1 if x > 0 else 0)
bc_final

,Group,Date,Close,Volume,Date_new,Increase
0,310000,2017-10-31 00:00:00,6124.68,0.134906,2017-10-31 00:00:00,0
1,310001,2017-10-31 00:01:00,6124.68,1.223548,2017-10-31 00:01:00,0
2,310002,2017-10-31 00:02:00,6124.69,3.294787,2017-10-31 00:02:00,1
3,310003,2017-10-31 00:03:00,6124.68,0.110868,2017-10-31 00:03:00,0
4,310004,2017-10-31 00:04:00,6124.69,0.087178,2017-10-31 00:04:00,1
...,...,...,...,...,...,...
1158642,271736,2017-11-27 17:36:00,9494.66,11.753007,2017-11-27 17:36:00,1
1158641,271737,2017-11-27 17:37:00,9496.96,1.201738,2017-11-27 17:37:00,1
1158640,271738,2017-11-27 17:38:00,9496.85,0.499032,2017-11-27 17:38:00,0
1158639,271739,2017-11-27 17:39:00,9496.85,0.007648,2017-11-27 17:39:00,0


In [56]:
bc_final_reset = bc_final.reset_index()
bc_final_reset

,index,Group,Date,Close,Volume,Date_new,Increase
0,0,310000,2017-10-31 00:00:00,6124.68,0.134906,2017-10-31 00:00:00,0
1,1,310001,2017-10-31 00:01:00,6124.68,1.223548,2017-10-31 00:01:00,0
2,2,310002,2017-10-31 00:02:00,6124.69,3.294787,2017-10-31 00:02:00,1
3,3,310003,2017-10-31 00:03:00,6124.68,0.110868,2017-10-31 00:03:00,0
4,4,310004,2017-10-31 00:04:00,6124.69,0.087178,2017-10-31 00:04:00,1
...,...,...,...,...,...,...,...
35924,1158642,271736,2017-11-27 17:36:00,9494.66,11.753007,2017-11-27 17:36:00,1
35925,1158641,271737,2017-11-27 17:37:00,9496.96,1.201738,2017-11-27 17:37:00,1
35926,1158640,271738,2017-11-27 17:38:00,9496.85,0.499032,2017-11-27 17:38:00,0
35927,1158639,271739,2017-11-27 17:39:00,9496.85,0.007648,2017-11-27 17:39:00,0


In [57]:
bc_final_red = bc_final_reset.iloc[:35880,:]
bc_final_red

,index,Group,Date,Close,Volume,Date_new,Increase
0,0,310000,2017-10-31 00:00:00,6124.68,0.134906,2017-10-31 00:00:00,0
1,1,310001,2017-10-31 00:01:00,6124.68,1.223548,2017-10-31 00:01:00,0
2,2,310002,2017-10-31 00:02:00,6124.69,3.294787,2017-10-31 00:02:00,1
3,3,310003,2017-10-31 00:03:00,6124.68,0.110868,2017-10-31 00:03:00,0
4,4,310004,2017-10-31 00:04:00,6124.69,0.087178,2017-10-31 00:04:00,1
...,...,...,...,...,...,...,...
35875,1158691,271647,2017-11-27 16:47:00,9543.96,12.665832,2017-11-27 16:47:00,0
35876,1158690,271648,2017-11-27 16:48:00,9551.59,8.480385,2017-11-27 16:48:00,1
35877,1158689,271649,2017-11-27 16:49:00,9573.30,18.366559,2017-11-27 16:49:00,1
35878,1158688,271650,2017-11-27 16:50:00,9575.32,13.193949,2017-11-27 16:50:00,1


In [58]:
bc_lstm = bc_final_reset[bc_final_reset.index % 60 == 0] # need only 35880 rows
bc_lstm = bc_lstm.iloc[1:,:]
bc_lstm

,index,Group,Date,Close,Volume,Date_new,Increase
60,60,310100,2017-10-31 01:00:00,6146.15,1.811310,2017-10-31 01:00:00,0
120,120,310200,2017-10-31 02:00:00,6140.00,0.127218,2017-10-31 02:00:00,0
180,180,310300,2017-10-31 03:00:00,6130.87,0.000000,2017-10-31 03:00:00,0
240,240,310400,2017-10-31 04:00:00,6132.81,0.042696,2017-10-31 04:00:00,0
300,300,310500,2017-10-31 05:00:00,6145.95,2.688068,2017-10-31 05:00:00,0
...,...,...,...,...,...,...,...
35640,1158926,271252,2017-11-27 12:52:00,9603.34,0.517777,2017-11-27 12:52:00,0
35700,1158866,271352,2017-11-27 13:52:00,9619.88,2.410453,2017-11-27 13:52:00,1
35760,1158806,271452,2017-11-27 14:52:00,9617.92,32.300756,2017-11-27 14:52:00,0
35820,1158746,271552,2017-11-27 15:52:00,9639.51,3.883218,2017-11-27 15:52:00,1


In [59]:
bc_lstm['Increase'] = bc_lstm['Close'].pct_change().apply(lambda x: 1 if x > 0 else 0)
bc_lstm

,index,Group,Date,Close,Volume,Date_new,Increase
60,60,310100,2017-10-31 01:00:00,6146.15,1.811310,2017-10-31 01:00:00,0
120,120,310200,2017-10-31 02:00:00,6140.00,0.127218,2017-10-31 02:00:00,0
180,180,310300,2017-10-31 03:00:00,6130.87,0.000000,2017-10-31 03:00:00,0
240,240,310400,2017-10-31 04:00:00,6132.81,0.042696,2017-10-31 04:00:00,1
300,300,310500,2017-10-31 05:00:00,6145.95,2.688068,2017-10-31 05:00:00,1
...,...,...,...,...,...,...,...
35640,1158926,271252,2017-11-27 12:52:00,9603.34,0.517777,2017-11-27 12:52:00,0
35700,1158866,271352,2017-11-27 13:52:00,9619.88,2.410453,2017-11-27 13:52:00,1
35760,1158806,271452,2017-11-27 14:52:00,9617.92,32.300756,2017-11-27 14:52:00,0
35820,1158746,271552,2017-11-27 15:52:00,9639.51,3.883218,2017-11-27 15:52:00,1


In [60]:
tweets_final_reset = tweets_final.reset_index()
tweets_final_reset

,index,Group,Tweet
0,0,310000,bitcoin cash cheapest on cryptopia btc market ...
1,1,310001,current bitcoin price gbp btc quantum attacks ...
2,2,310002,rt participate in the dalecoin second round bo...
3,3,310003,rt what is bitconnect quick start guide to bit...
4,4,310004,marketalert last top price change avg of the t...
...,...,...,...
35924,10316,271736,reason to consider ico this holiday season we ...
35925,10317,271737,rt crypto current price of bitcoin is rt bitco...
35926,10318,271738,stickac latest firmware is no different to old...
35927,10319,271739,here's maybe strange question but think it's o...


In [61]:
tweets_lstm = tweets_final_reset.iloc[:35880,:]
tweets_lstm

,index,Group,Tweet
0,0,310000,bitcoin cash cheapest on cryptopia btc market ...
1,1,310001,current bitcoin price gbp btc quantum attacks ...
2,2,310002,rt participate in the dalecoin second round bo...
3,3,310003,rt what is bitconnect quick start guide to bit...
4,4,310004,marketalert last top price change avg of the t...
...,...,...,...
35875,10267,271647,rt kohls cash the new bitcoin invest now befor...
35876,10268,271648,rt short history of bitcoin the number one inv...
35877,10269,271649,rt bitcoin could lose of it's value and it wou...
35878,10270,271650,opinion the chance of bitcoin crash is greater...


In [62]:
x_list = tweets_lstm['Tweet'].tolist()
y_list = bc_final_red['Increase'].tolist()

x_train_list = x_list[:28680]
y_train_list = y_list[:28680]

In [63]:
x_test_list = x_list[28680:]
y_test_list = y_list[28680:]
print(len(x_test_list))
print(len(y_test_list))

7200
7200


In [64]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as  np

vectorizer = CountVectorizer(max_features = 30000, dtype = np.float32)
vectorizer.fit(x_train_list)


CountVectorizer(dtype=<class 'numpy.float32'>, max_features=30000)

In [65]:
print(len(vectorizer.vocabulary_))

30000


In [66]:
# encode document
vector = vectorizer.transform(x_train_list)
# summarize encoded vector
print(vector.shape)
vector_array = vector.toarray()
print(vector_array)

(28680, 30000)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [67]:
vector

<28680x30000 sparse matrix of type '<class 'numpy.float32'>'
	with 11221334 stored elements in Compressed Sparse Row format>

In [68]:
y_train = np.array(y_train_list).astype('float32')
y_train

array([0., 0., 1., ..., 1., 0., 1.], dtype=float32)

In [69]:
# encode document
vector_test = vectorizer.transform(x_test_list)
# summarize encoded vector
print(vector_test.shape)
vector_test_array = vector_test.toarray()
print(vector_test_array)

(7200, 30000)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [70]:
y_test = np.array(y_test_list).astype('float32')
y_test

array([1., 0., 1., ..., 1., 1., 1.], dtype=float32)

In [ ]:
#### Model with 2 Dense layers stacked together

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(16, input_dim=60000, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(vector_array, y_train, batch_size = 32, epochs=10, verbose = 1)

Epoch 1/10
897/897 [==============================] - 12s 13ms/step - loss: 0.6570 - accuracy: 0.6307
Epoch 2/10
897/897 [==============================] - 12s 13ms/step - loss: 0.6224 - accuracy: 0.6381
Epoch 3/10
897/897 [==============================] - 12s 13ms/step - loss: 0.5583 - accuracy: 0.7041
Epoch 4/10
897/897 [==============================] - 12s 13ms/step - loss: 0.4882 - accuracy: 0.7604
Epoch 5/10
897/897 [==============================] - 12s 13ms/step - loss: 0.4246 - accuracy: 0.8028
Epoch 6/10
897/897 [==============================] - 12s 13ms/step - loss: 0.3740 - accuracy: 0.8350
Epoch 7/10
897/897 [==============================] - 12s 13ms/step - loss: 0.3409 - accuracy: 0.8553
Epoch 8/10
897/897 [==============================] - 12s 13ms/step - loss: 0.3031 - accuracy: 0.8745
Epoch 9/10
897/897 [==============================] - 12s 13ms/step - loss: 0.2792 - accuracy: 0.8867
Epoch 10/10
897/897 [==============================] - 12s 13ms/step - loss: 0.271

In [ ]:
model.evaluate(vector_test_array, y_test, verbose = 1)   

225/225 [==============================] - 1s 5ms/step - loss: 2.4131 - accuracy: 0.5200


[2.413097858428955, 0.5199999809265137]

In [ ]:
model.metrics_names

['loss', 'accuracy']

In [ ]:
#### We can see a lot of overfitting in above model training accuracy is very high as compared to validation accuracy
#### So we will use Dropout layers and check

In [71]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(16, input_dim=30000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                480016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 480,305
Trainable params: 480,305
Non-trainable params: 0
_________________________________________________________________


In [72]:
model.fit(vector_array, y_train, batch_size = 32, epochs=10, verbose = 1)

Epoch 1/10
897/897 [==============================] - 7s 6ms/step - loss: 0.6640 - accuracy: 0.6290
Epoch 2/10
897/897 [==============================] - 5s 6ms/step - loss: 0.6573 - accuracy: 0.6307
Epoch 3/10
897/897 [==============================] - 5s 6ms/step - loss: 0.6493 - accuracy: 0.6306
Epoch 4/10
897/897 [==============================] - 5s 6ms/step - loss: 0.6336 - accuracy: 0.6306
Epoch 5/10
897/897 [==============================] - 5s 6ms/step - loss: 0.6169 - accuracy: 0.6307
Epoch 6/10
897/897 [==============================] - 5s 5ms/step - loss: 0.5980 - accuracy: 0.6306
Epoch 7/10
897/897 [==============================] - 5s 6ms/step - loss: 0.5807 - accuracy: 0.6307
Epoch 8/10
897/897 [==============================] - 5s 6ms/step - loss: 0.5665 - accuracy: 0.6307
Epoch 9/10
897/897 [==============================] - 5s 5ms/step - loss: 0.5494 - accuracy: 0.6307
Epoch 10/10
897/897 [==============================] - 5s 5ms/step - loss: 0.5385 - accuracy: 0.6355

In [73]:
model.evaluate(vector_test_array, y_test, verbose = 1)

225/225 [==============================] - 1s 4ms/step - loss: 1.3408 - accuracy: 0.5067


[1.340752124786377, 0.5066666603088379]

In [74]:
model.metrics_names

['loss', 'accuracy']